# Plane Tracker — Visual MVP (OpenSky + Jupyter)

This notebook helps you **visually** iterate on the plane-tracking MVP and includes a **one-click cell** to fetch the **nearest flight number**.

**Workflow:**  
1. Set your **latitude/longitude** and **radius** in the config cell.  
2. Run the **Quick Fetch Nearest** cell — it computes the bounding box, calls OpenSky, and prints the nearest callsign.  
3. (Optional) Use the refresh/table/map cells for exploration.


In [ ]:
# === 0) Setup: installs (run once if needed) ===
# If needed, uncomment and run once in your venv:
# !pip install requests pandas matplotlib


In [15]:
# === 1) Configuration ===
# 👉 Replace these with your actual coordinates (decimal degrees).
MY_LAT = 33.95812
MY_LON = -118.39025
RADIUS_KM = 10.0   # search radius (km)

# Minimum altitude to consider (meters). Ignore aircraft "on ground" and below this altitude.
MIN_ALT_M = 50

print("Config set. Edit MY_LAT / MY_LON / RADIUS_KM as needed.")

Config set. Edit MY_LAT / MY_LON / RADIUS_KM as needed.


In [16]:
# === 2) Utilities (distance, bbox, nearest) ===
import math, requests, pandas as pd

EARTH_RADIUS_KM = 6371.0

def haversine_km(lat1, lon1, lat2, lon2):
    f1, f2 = math.radians(lat1), math.radians(lat2)
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(f1)*math.cos(f2)*math.sin(dlon/2)**2
    return 2 * EARTH_RADIUS_KM * math.asin(math.sqrt(a))

def bbox_from_center(lat, lon, radius_km):
    dlat = radius_km / 111.0
    dlon = radius_km / (111.0 * max(0.0001, math.cos(math.radians(lat))))
    lat_min, lat_max = lat - dlat, lat + dlat
    lon_min, lon_max = lon - dlon, lon + dlon
    return lat_min, lat_max, lon_min, lon_max

def fetch_opensky(lat_min, lat_max, lon_min, lon_max, extended=True, timeout=10):
    params = {"lamin": f"{lat_min:.5f}", "lamax": f"{lat_max:.5f}",
              "lomin": f"{lon_min:.5f}", "lomax": f"{lon_max:.5f}"}
    if extended:
        params["extended"] = 1
    url = "https://opensky-network.org/api/states/all"
    r = requests.get(url, params=params, timeout=timeout)
    r.raise_for_status()
    return r.json(), r.url

def tidy_states(data, lat0, lon0, min_alt_m=MIN_ALT_M, radius_km=RADIUS_KM):
    states = data.get("states", []) or []
    rows = []
    for s in states:
        callsign = (s[1] or "").strip()
        lon = s[5]; lat = s[6]
        if lon is None or lat is None:
            continue
        on_ground = bool(s[8]) if s[8] is not None else False
        geo_alt_m = s[13] if len(s) > 13 else None
        baro_alt_m = s[7]
        alt_m = geo_alt_m if geo_alt_m is not None else baro_alt_m
        if alt_m is None:
            continue
        dist_km = haversine_km(lat0, lon0, lat, lon)
        if dist_km > radius_km or on_ground or alt_m < min_alt_m:
            continue
        rows.append({"callsign": callsign if callsign else "(no callsign)",
                     "icao24": s[0], "lat": lat, "lon": lon,
                     "alt_m": int(alt_m), "dist_km": dist_km})
    df = pd.DataFrame(rows)
    if not df.empty:
        df = df.sort_values("dist_km").reset_index(drop=True)
    return df

def get_nearest_callsign(lat0, lon0, radius_km=RADIUS_KM, min_alt_m=MIN_ALT_M):
    lat_min, lat_max, lon_min, lon_max = bbox_from_center(lat0, lon0, radius_km)
    data, url = fetch_opensky(lat_min, lat_max, lon_min, lon_max, extended=True)
    df = tidy_states(data, lat0, lon0, min_alt_m=min_alt_m, radius_km=radius_km)
    return df, url, (lat_min, lat_max, lon_min, lon_max)

In [24]:
# === 3) Quick Fetch Nearest (one-click) ===
try:
    df, requested_url, bbox = get_nearest_callsign(MY_LAT, MY_LON, RADIUS_KM, MIN_ALT_M)
    lat_min, lat_max, lon_min, lon_max = bbox
    print("Requested URL:", requested_url)
    print(f"Bounding box: lat ∈ [{lat_min:.5f}, {lat_max:.5f}], lon ∈ [{lon_min:.5f}, {lon_max:.5f}]")
    if df.empty:
        print("No airborne aircraft found in range. Try increasing RADIUS_KM, wait a bit, or check your coordinates.")
    else:
        nearest_callsign = df.iloc[0]["callsign"]
        print("Nearest callsign:", nearest_callsign)
        display(df.head(5))
except Exception as e:
    print("Error during fetch:", e)

Requested URL: https://opensky-network.org/api/states/all?lamin=33.86803&lamax=34.04821&lomin=-118.49886&lomax=-118.28164&extended=1
Bounding box: lat ∈ [33.86803, 34.04821], lon ∈ [-118.49886, -118.28164]
Nearest callsign: DAL634


,callsign,icao24,lat,lon,alt_m,dist_km
0,DAL634,a36898,33.9532,-118.3909,76,0.550354
1,UAL1498,a12799,33.9545,-118.3780,129,1.199406
2,BAW21B,40621c,33.9383,-118.3738,60,2.675723
3,AAL821,abdb35,33.9386,-118.3713,76,2.786853
4,N32JE,a36cc4,33.9386,-118.3704,83,2.839659
